# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В каждой строке таблицы — данные о потенциальных заёмщиках.Колонки описывают разные признаки заёмщиков: их возраст, пол, образование, семейное положение, количество детей, тип занятости, ежемесячный доход, общий трудовой стаж, наличие задолженностей по возврату кредитов и цель получения кредита. Так же в таблице есть колонки-идентификаторы: идентификатор образования и идентификатор семейного положения.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, некотрые колонки заполнены в разном регистре. А так же есть проблемы с типом данных в некоторых колонках.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data.isna().sum()

data_transformed = data.groupby('income_type')['total_income'].transform('mean') #заполнили пропуски в total_income

data['total_income'] = data['total_income'].fillna(data_transformed)

data['days_employed'] = data['days_employed'].abs() # избавились от отрицательных значений в days_employed

data_transformed = data.groupby('income_type')['days_employed'].transform('mean') #заполнили пропуски в days_employed

data['days_employed'] = data['days_employed'].fillna(data_transformed)  

data['dob_years'] = data['dob_years'].abs() # избавились от отрицательных значений в dob_years

#display(data.groupby('income_type')['dob_years'].mean())
#display(data[data['dob_years'] <= 0].count())
def age_transformed(row):
    dob_years = row['dob_years']
    income_type = row['income_type']
    
    if dob_years == 0:
        if income_type == 'безработный':
            return 38
        if income_type == 'в декрете' or income_type == 'компаньон' or income_type == 'сотрудник':
            return 39
        if income_type == 'госслужащий':
            return 40
        if income_type == 'пенсионер':
            return 59
        if income_type == 'предприниматель':
            return 42
        if income_type == 'студент':
            return 22
    return row['dob_years']    
data['dob_years'] = data.apply(age_transformed, axis=1)   
#display(data[data['dob_years'] <= 0].count()) проверка показала отсутсвие нулей в возрасте
#display(data.groupby('gender').count())
data.drop(data[data.gender == 'XNA'].index, inplace=True) #удалили некорректное значение
#display(data.groupby('gender').count()) #проверка прошла успешно
data['children'] = data['children'].abs() # избавились от отрицательных значений в children
data['education'] = data['education'].str.lower() #привели значение колонки education к нижнему регистру
display(data)


        
        


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

Мы обнаружили, что количество пропусков в двух колоках совпадает, а именно: days_employed (общий трудовой стаж) и total_income (ежемесячный доход). Таких строк в датафрейме 2174, что составляет прмерно 10% от всех данных, следовательно мы не можем просто взять и удалить эти пропуски. Заменим пропуски на средние величины, сгруппировав по income_type. После проделанных манипуляций пропуски в строках исчезли, но проблемы в данных остались.
Первое, что явно бросается в глаза - отричательные величины в колонке days_employed (общий трудовой стаж). Возьмем всю колонку по модулю. Такая проблема появилась, вероятнее всего, из-за того, что только у пенсионеров есть конечная дата трудового стажа, а у тех кто работает в настоящее время - нет.
В колонке возраста есть значения 0 ( а именно 101 значение), что является ошибкой в данных, возможно человеческий фактор. Заполним её средним значением по группам.
В колонке gender обнаружено одно некорректное значение XNA. Т.к. эта колонка нужна для исследований и такое значение всего одно, то я решила удалить строку с этим значением.
В колонке children модулем избавились от отрицательных значений.
В колонке education значения записаны в разных регистрах. Это 100% человеческий фактор, приведем все к одному виду для дальнейшего исследования.

### Замена типа данных


In [3]:
data['days_employed'] = data['days_employed'].astype('int')


**Вывод**

Столбец days_employed показывает общий трудовой стаж в днях. Поэтому данный столбец лучше перевести из вещественного в целочисленный, т.к. нас интересует целое количество дней.

### Обработка дубликатов

In [4]:
data.duplicated().sum()
data_duplicated = data[data.duplicated()]
#display(data_duplicated)
data = data.drop_duplicates().reset_index(drop=True)
#display(data_duplicated)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Первым делом я удалила из датафрейма явные дубликаты с помощью функции drop_duplicates().
Теперь необходимо проверить датафрейм на неявные дубликаты. Для этого необходимо лемматизировать столбец purpose.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem() 

def lem_purpose(column):
    lemmas = m.lemmatize(column)
    return lemmas

data['purpose_lemm'] = data['purpose'].apply(lem_purpose)

display(data.head())


from collections import Counter

count_lemm = []

for ind in range(len(data['purpose_lemm'])):
    count_lemm.extend(data['purpose_lemm'][ind])

print(Counter(count_lemm))



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]"


Counter({' ': 33569, '\n': 21453, 'недвижимость': 6350, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


**Вывод**

С помощью библиотеки pymystem3 лемматизировали столбец purpose (цель кредита). Функцией Counter подсчитали колличество лемм, чтобы в дальнейшем можно было категоризировать данные по цели кредита.

### Категоризация данных

In [6]:
def purpose_category(row): #категоризация по цели кредита
    
    purpose_lemm = row['purpose_lemm']
    
    if (('жилье' in purpose_lemm) or ('недвижимость' in purpose_lemm)) and ('строительство' not in purpose_lemm):
        return 'покупка недвижимости'
    if (('жилье' in purpose_lemm) or ('недвижимость' in purpose_lemm)) and ('строительство' in purpose_lemm):
        return 'строительство недвижимости'
    if 'автомобиль' in purpose_lemm:
        return 'покупка автомобиля'
    if 'образование' in purpose_lemm:
        return 'образование'
    if 'свадьба' in purpose_lemm:
        return 'свадьба'
    return purpose_lemm

data['purpose_category'] = data.apply(purpose_category, axis=1)

def children_category(row): #категоризация по наличию детей
    
    children = row['children']
    
    if children == 0:
        return 'бездетные'
    return 'есть дети'

data['children_category'] = data.apply(children_category, axis=1)

#print(data['total_income'].max()) #2265604.028722744
#print(data['total_income'].min()) #20667.26379327158
#print(data['total_income'].mean()) #167430.36925500483
#print(data['total_income'].quantile(0.25)) #107620.94810490453
#print(data['total_income'].quantile(0.75)) #202417.46146177707

def income_category(row): #категоризация по доходам
    
    total_income = row['total_income']
    
    if total_income >= data['total_income'].quantile(0.75):
        return 'высокий доход'
    if total_income >= data['total_income'].quantile(0.25):
        return 'средний доход'
    return 'низкий доход'

data['income_category'] = data.apply(income_category, axis=1)

display(data.head())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm,purpose_category,children_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",покупка недвижимости,есть дети,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",покупка автомобиля,есть дети,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",покупка недвижимости,бездетные,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,бездетные,средний доход


**Вывод**

Для ответов на поставленные вопросы данные необходимо было категоризировать в нескольких направлениях: наличие/отсутсвие детей, цель кредитования, а так же уровень дохода.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
data_pivot_children = data.pivot_table(index=['children_category'], columns='debt', values='income_type', aggfunc='count')

data_pivot_children['no_debt'] = data_pivot_children[0]*100/(data_pivot_children[0] + data_pivot_children[1])
data_pivot_children['yes_debt'] = data_pivot_children[1]*100/(data_pivot_children[0] + data_pivot_children[1])

display(data_pivot_children) 

debt,0,1,no_debt,yes_debt
children_category,,,,
бездетные,13027,1063,92.455642,7.544358
есть дети,6685,678,90.791797,9.208203


**Вывод**

По данным сводной таблицы бездетные люди реже имели просрочки по кредитам

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
data_pivot_family = data.pivot_table(index=['family_status'], columns='debt', values='income_type', aggfunc= 'count')

data_pivot_family['no_debt'] = data_pivot_family[0]*100/(data_pivot_family[0] + data_pivot_family[1])
data_pivot_family['yes_debt'] = data_pivot_family[1]*100/(data_pivot_family[0] + data_pivot_family[1])    

display(data_pivot_family)


debt,0,1,no_debt,yes_debt
family_status,,,,
Не женат / не замужем,2536,274,90.249110,9.750890
в разводе,1110,85,92.887029,7.112971
вдовец / вдова,896,63,93.430657,6.569343
гражданский брак,3762,388,90.650602,9.349398
женат / замужем,11408,931,92.454818,7.545182


**Вывод**

Как видно из сводной таблицы - вдовы/вдовцы имеют самый низкий процент просрочки, по отношению к людям с другим семейным положением. Незамужние граждане чаще других пропускают платежи по кредитам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data_pivot_income = data.pivot_table(index=['income_category'], columns='debt', values='income_type', aggfunc='count')

data_pivot_income['no_debt'] = data_pivot_income[0]*100/(data_pivot_income[0] + data_pivot_income[1])
data_pivot_income['yes_debt'] = data_pivot_income[1]*100/(data_pivot_income[0] + data_pivot_income[1])    

display(data_pivot_income)

debt,0,1,no_debt,yes_debt
income_category,,,,
высокий доход,5034,375,93.067110,6.932890
низкий доход,4936,427,92.038038,7.961962
средний доход,9742,939,91.208688,8.791312


**Вывод**

Зависимость, определенно есть, и она линейная: чем выше доход, тем ниже просрочек про кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
data_pivot_purpose = data.pivot_table(index=['purpose_category'], columns='debt', values='income_type', aggfunc='count')

data_pivot_purpose['no_debt'] = data_pivot_purpose[0]*100/(data_pivot_purpose[0] + data_pivot_purpose[1])
data_pivot_purpose['yes_debt'] = data_pivot_purpose[1]*100/(data_pivot_purpose[0] + data_pivot_purpose[1])    

display(data_pivot_purpose)


debt,0,1,no_debt,yes_debt
purpose_category,,,,
образование,3643,370,90.779965,9.220035
покупка автомобиля,3903,403,90.640966,9.359034
покупка недвижимости,8294,638,92.857143,7.142857
свадьба,2138,186,91.996558,8.003442
строительство недвижимости,1734,144,92.332268,7.667732


**Вывод**

Цель кредита очень мало влияет на возврат в срок ( общая погрешность примерно 1,5 %). Однако, кредиты на покупку недвижимости заемщики отдают регулярнее, чем кредиты на покупку автомобиля.

## Шаг 4. Общий вывод

Мы провели полную предобработку данных, чтобы дать ответы на интересующие заказчика вопросы:

Есть ли зависимость между наличием детей и возвратом кредита в срок?
Определенно, есть. Как показало исследование - бездетные заёмщики реже имеют проблемы с возвратом кредита в срок.

Есть ли зависимость между семейным положением и возвратом кредита в срок?
И, снова, да, зависимость есть. Вдовы/вдовцы имеют самый низкий процент просрочки, по отношению к людям с другим семейным положением. Незамужние граждане чаще других пропускают платежи по кредитам.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Зависимость есть, и имеет линейный вид, т.е. чем выше доход, тем меньше просрочек по кредитам.

Как разные цели кредита влияют на его возврат в срок?
Тут получилось неоднозначно. Разбег не большой ( всего 1.5%). Но кредиты на покупку недвижимости заемщики отдают регулярнее, чем кредиты на покупку автомобиля.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.